In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from PIL import Image
import math
import os
import time

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 512
low_dim = 128
lr = 0.03
batch_m = 1
batch_t = 100 # 1会爆炸，更不用说更小的了，10也会爆炸
start_epoch = 0

In [3]:
class CIFAR10Instance(datasets.CIFAR10):
    """CIFAR10Instance Dataset.
    """
    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img)
        
        if self.target_transform is not None:
            target = self.target_transform(target)
            
        if self.transform is not None:
            img1 = self.transform(img)
            if self.train:
                img2 = self.transform(img)

        if self.train:
            return img1, img2, target, index
        else:
            return img1, target, index

In [4]:
import os

# 获取当前脚本的绝对路径
current_dir = os.path.dirname(os.path.abspath("Untitled.ipynb"))
# 构建 a/b/data 文件夹的路径
data_dir = os.path.abspath(os.path.join(current_dir, '..', '1. Unsupervised Feature Learning via Non-parameteric Instance Discrimination', 'data'))
print(data_dir)

print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(size=32, scale=(0.2,1.)),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0),
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# 使用构建的路径来访问数据集
trainset = CIFAR10Instance(root=data_dir, train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last =True)

testset = CIFAR10Instance(root=data_dir, train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=100, shuffle=False, drop_last =True)

ndata = len(trainset)

C:\Users\20703\d2l-zh\pytorch\MyExercises\对比学习\1. Unsupervised Feature Learning via Non-parameteric Instance Discrimination\data
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [5]:
# test
# from ImageTensor import tensor_to_image

# import torch
# import matplotlib.pyplot as plt

# # 定义可视化函数
# def visualize_tensor(tensor, indices, title=""):
#     fig, axes = plt.subplots(1, len(indices), figsize=(9, 3))
#     if len(indices) == 1:
#         axes = [axes]  # 将单个AxesSubplot对象包装成一个列表
#     for ax, idx in zip(axes, indices):
#         image = tensor[idx, 0].detach().cpu().numpy()
#         ax.imshow(image, cmap='gray')
#         ax.set_title(f"Image {idx+1}")
#         ax.axis('off')
#     fig.suptitle(title)
#     plt.show()

# 在训练过程中调用
for batch_idx, (inputs1, inputs2, target, indexes) in enumerate(trainloader):
    print(inputs1.shape, inputs2.shape, target.shape, indexes.shape)
    print(target, indexes)
    # visualize_tensor(inputs, [1, 2, 3])
    # 将 PyTorch Tensor 转换为图像
    # inputs1_image = tensor_to_image(inputs1[1].permute(1, 2, 0))
    # inputs2_image = tensor_to_image(inputs2[1].permute(1, 2, 0))
    # inputs2_image.show()
    break

torch.Size([512, 3, 32, 32]) torch.Size([512, 3, 32, 32]) torch.Size([512]) torch.Size([512])
tensor([9, 8, 1, 1, 1, 0, 0, 9, 1, 6, 0, 9, 7, 0, 8, 0, 6, 9, 4, 0, 0, 8, 5, 8,
        2, 4, 3, 6, 4, 1, 8, 1, 1, 8, 8, 4, 6, 1, 1, 5, 0, 1, 9, 1, 1, 3, 7, 6,
        8, 2, 0, 1, 9, 5, 9, 0, 2, 1, 6, 0, 3, 9, 9, 2, 6, 4, 6, 9, 0, 7, 7, 5,
        3, 7, 2, 2, 7, 2, 8, 6, 3, 9, 3, 3, 7, 5, 6, 4, 0, 1, 0, 3, 1, 3, 6, 4,
        6, 2, 2, 2, 7, 1, 3, 2, 9, 4, 5, 5, 5, 0, 7, 4, 2, 6, 4, 6, 4, 0, 6, 3,
        4, 7, 0, 6, 9, 7, 7, 8, 5, 3, 9, 2, 9, 5, 1, 5, 5, 2, 8, 2, 1, 2, 0, 1,
        2, 9, 1, 9, 3, 5, 6, 9, 2, 2, 2, 1, 0, 8, 2, 0, 5, 6, 0, 5, 8, 9, 5, 7,
        7, 7, 8, 3, 9, 7, 1, 5, 9, 5, 2, 5, 7, 4, 1, 1, 4, 2, 8, 3, 8, 8, 0, 4,
        8, 1, 7, 2, 0, 1, 8, 1, 2, 7, 3, 5, 2, 1, 8, 1, 5, 1, 1, 6, 1, 7, 3, 2,
        7, 4, 4, 4, 0, 7, 2, 6, 0, 5, 5, 4, 0, 4, 5, 7, 6, 2, 3, 9, 7, 9, 7, 5,
        9, 8, 9, 7, 2, 2, 5, 7, 7, 7, 0, 7, 8, 7, 5, 1, 7, 5, 5, 4, 3, 7, 9, 9,
        9, 6, 6, 3, 0, 8, 

In [6]:
print('==> Building model..')
net = torchvision.models.resnet18(num_classes=low_dim)  # Using ResNet18 with the low_dim output

from BatchAverage import BatchCriterion

"""
parser.add_argument('--batch-t', default=0.1, type=float,
                    metavar='T', help='temperature parameter for softmax')
parser.add_argument('--batch-m', default=1, type=float,
                    metavar='N', help='m for negative sum')

"def __init__(self, negM, T, batchSize):"

self.negM = negM:
这里 negM 是一个参数，代表负样本的加权系数。它用于控制负样本的权重，在损失计算中会用到。

self.T = T:
T 是温度参数，用于缩放内积值，使得 softmax 函数更加平滑。较小的 T 会使得模型对小的差异更加敏感，较大的 T 会使得模型的输出更加均匀。
"""
# define loss function: inner product loss within each mini-batch
criterion = BatchCriterion(batch_m, batch_t, batch_size)

net.to(device)
criterion.to(device)

==> Building model..


BatchCriterion()

In [7]:
# define optimizer
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

In [8]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed at 120, 160 and 200"""
    lr = 0.03
    if epoch >= 120 and epoch < 160:
        lr = args.lr * 0.1
    elif epoch >= 160 and epoch <200:
        lr = args.lr * 0.05
    elif epoch >= 200:
        lr = args.lr * 0.01
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr   
    # writer.add_scalar('lr',  lr, epoch)

In [9]:
class AverageMeter(object):
    """Computes and stores the average and current value""" 
    def __init__(self):
        self.reset()
                   
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0 

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [10]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    adjust_learning_rate(optimizer, epoch)
    train_loss = AverageMeter()
    data_time = AverageMeter()
    batch_time = AverageMeter()

    # switch to train mode
    net.train()

    end = time.time()
    for batch_idx, (inputs1, inputs2, _, indexes) in enumerate(trainloader):#
        # inputs1.shape, inputs2.shape, target.shape, indexes.shape
        # torch.Size([64, 3, 32, 32]) torch.Size([64, 3, 32, 32]) torch.Size([64]) torch.Size([64])
        data_time.update(time.time() - end)

        inputs1, inputs2, indexes = inputs1.to(device), inputs2.to(device), indexes.to(device)
        
        inputs = torch.cat((inputs1,inputs2), 0) # (128, 3, 32, 32)
        optimizer.zero_grad()

        features = net(inputs) # features.shape = (128, 128)
        # print("###", features.shape, "###", indexes.shape, "###") # ### torch.Size([128, 128]) ### torch.Size([64]) ### batch_size设置的是64
        loss = criterion(features, indexes) # features 是64原数据+64增强数据的特征，indexes是64原数据对应的index（不是target，label）

        loss.backward()
        optimizer.step()
        
        train_loss.update(loss.item(), inputs.size(0))         
        
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        if batch_idx%1 ==0:
            print('Epoch: [{}][{}/{}] '
                  'Time: {batch_time.val:.3f} ({batch_time.avg:.3f}) '
                  'Data: {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Loss: {train_loss.val:.4f} ({train_loss.avg:.4f})'.format(
                  epoch, batch_idx, len(trainloader), batch_time=batch_time, data_time=data_time, train_loss=train_loss))
    # add log
    # writer.add_scalar('loss',  train_loss.avg, epoch)

In [11]:
for epoch in range(start_epoch, start_epoch+301):
    train(epoch)


Epoch: 0


D:\Miniconda\envs\DL_learning\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
D:\Miniconda\envs\DL_learning\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch: [0][0/97] Time: 3.441 (3.441) Data: 1.710 (1.710) Loss: 7.9117 (7.9117)
Epoch: [0][1/97] Time: 2.315 (2.878) Data: 1.383 (1.546) Loss: 7.9088 (7.9102)
Epoch: [0][2/97] Time: 2.181 (2.646) Data: 1.097 (1.396) Loss: 7.9158 (7.9121)
Epoch: [0][3/97] Time: 2.430 (2.592) Data: 1.501 (1.423) Loss: 7.9042 (7.9101)
Epoch: [0][4/97] Time: 1.900 (2.454) Data: 0.969 (1.332) Loss: 7.9079 (7.9097)
Epoch: [0][5/97] Time: 1.750 (2.336) Data: 0.824 (1.247) Loss: 7.9094 (7.9096)
Epoch: [0][6/97] Time: 1.988 (2.287) Data: 0.818 (1.186) Loss: 7.8974 (7.9079)
Epoch: [0][7/97] Time: 2.044 (2.256) Data: 1.112 (1.177) Loss: 7.8886 (7.9055)
Epoch: [0][8/97] Time: 2.149 (2.244) Data: 1.088 (1.167) Loss: 7.8855 (7.9033)
Epoch: [0][9/97] Time: 2.049 (2.225) Data: 1.109 (1.161) Loss: 7.8769 (7.9006)
Epoch: [0][10/97] Time: 2.010 (2.205) Data: 0.820 (1.130) Loss: 7.8710 (7.8979)
Epoch: [0][11/97] Time: 2.009 (2.189) Data: 0.810 (1.103) Loss: 7.8830 (7.8967)
Epoch: [0][12/97] Time: 1.729 (2.154) Data: 0.794 

KeyboardInterrupt: 